In [ ]:
## Task 3
import requests
from bs4 import BeautifulSoup

star_to_int = {"zero":0, "one":1, "two":2, "three":3, "four":4, "five":5}
base_url = "https://books.toscrape.com/catalogue/"

page_link = f"https://books.toscrape.com/catalogue/page-1.html"
res = requests.get(page_link)
soup = BeautifulSoup(res.text, "html.parser")
pages = int(soup.find("li", attrs={"class":"current"}).text.replace("\n", " ").strip().split()[-1])


with open("books.csv", mode='w', encoding='utf-8') as fd:
    fd.write("title book,price,is avalible,rating,link\n") # header
    for page in range(1, pages+1): # pageination
        print(f"collecting data of page {page}")
        page_link = f"https://books.toscrape.com/catalogue/page-{page}.html"
        res = requests.get(page_link)
        soup = BeautifulSoup(res.text, "html.parser")
        ol_tag = soup.find("ol", attrs={"class":"row"})
        books_tag = ol_tag.find_all("li")
        for i in range(len(books_tag)):
            title_book = books_tag[i].find("h3").find("a").attrs['title'] # soln 1
            price_tag = books_tag[i].find("div", attrs={"class":"product_price"}).find("p", attrs={"class":"price_color"})
            price = float(price_tag.text[2:])
            stock_tag = books_tag[i].find("div", attrs={"class":"product_price"}).find("p", attrs={"class":"instock availability"})
            stock_avp = stock_tag.text.replace("\n", " ").strip().lower()
            stock_avaliblity = False
            if stock_avp == 'in stock':
                stock_avaliblity = True
            rate_title = books_tag[i].find("p", attrs={"class":"star-rating"}).attrs["class"][-1].lower()
        #     print(rate_title)
            rate = star_to_int[rate_title]
            url = base_url + books_tag[i].find_all("a")[-1].attrs['href']
#             res_book = requests.get(url)
#             .. more details
            fd.write(f"\"{title_book}\",{price},{stock_avaliblity},{rate},{url}\n")